# Extracting gifs and videos from Tensorbaord logging for Airfoil convergence

* Use tensorboard's `event_accumulator`


In [2]:
from tensorboard.backend.event_processing import event_accumulator
import cv2
import numpy as np
import os

In [3]:
# Tensorboard event file for each case

event_paths = [
    "./trained_gan/2_10/logs/events.out.tfevents.1748680094.mars.ihs.uni-stuttgart.de.10977.0",
    "./trained_gan/2_10/logs/events.out.tfevents.1748692205.snickers.ihs.uni-stuttgart.de.1735.0"
]

In [5]:
# Image tags
image_tag = 'Generated HydroFoils'

# custom folder to save extracted images
extracted_dir = "./results/2_10/frames"
annotated_dir = "./results/2_10/frames_annotated"

In [6]:
# Check if directory exists
os.makedirs(extracted_dir, exist_ok=True)
os.makedirs(annotated_dir, exist_ok=True)


In [7]:
for event_path in event_paths:
    ea = event_accumulator.EventAccumulator(event_path)
    ea.Reload()
    
    if image_tag not in ea.Tags()['images']:
        print(f"tag '{image_tag}' not found in {event_path}. Skipping.")
        continue
    # Extract images
    images = ea.Images(image_tag)

    for img_event in images:
        step = img_event.step
        img_array = np.frombuffer(img_event.encoded_image_string, dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        cv2.imwrite(f"{extracted_dir}/hydrofoil_epoch_{step:05d}.png", img)
print("Images extracted successfully.")

Images extracted successfully.


## Upper part is still pending. I need to save all logging images for 1 training, irrespective of its size.

# Model Architecture paramter size?

In [20]:
# load teh model 
from hydFoilGAN.gan import *


In [21]:
# Hyperparamters
import torch
LATENT_DIM = 2
NOISE_DIM = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load model 
checkpoint = torch.load("./trained_gan/2_10/checkpoints/model_epoch_10000.pth")

#generator model
model_G = Generator(latent_dim=LATENT_DIM,
                    noise_dim=NOISE_DIM).to(device)
#discriminator model
model_D = Discriminator(latent_dim=LATENT_DIM).to(device)

In [22]:
# load generator and discriminator weights
model_G.load_state_dict(checkpoint['model_G_state_dict'])
model_D.load_state_dict(checkpoint['model_D_state_dict'])

<All keys matched successfully>

In [25]:
# torchsummary
import torch
from torchsummary import summary

In [30]:
# generator summary Linear part
summary(model_G.gen1, input_size=(LATENT_DIM+NOISE_DIM,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1024]          13,312
       BatchNorm1d-2                 [-1, 1024]           2,048
         LeakyReLU-3                 [-1, 1024]               0
            Linear-4                 [-1, 3072]       3,148,800
       BatchNorm1d-5                 [-1, 3072]           6,144
         LeakyReLU-6                 [-1, 3072]               0
Total params: 3,170,304
Trainable params: 3,170,304
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.09
Params size (MB): 12.09
Estimated Total Size (MB): 12.19
----------------------------------------------------------------


In [29]:
# Summarize the conv transpose part (gen2)
depth_cpw = 32*8
dim_cpw = int((31 + 2)/8)
summary(model_G.gen2, input_size=(depth_cpw, dim_cpw, 3))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1            [-1, 128, 8, 3]         393,344
       BatchNorm2d-2            [-1, 128, 8, 3]             256
         LeakyReLU-3            [-1, 128, 8, 3]               0
   ConvTranspose2d-4            [-1, 64, 16, 3]          98,368
       BatchNorm2d-5            [-1, 64, 16, 3]             128
         LeakyReLU-6            [-1, 64, 16, 3]               0
   ConvTranspose2d-7            [-1, 32, 32, 3]          24,608
       BatchNorm2d-8            [-1, 32, 32, 3]              64
         LeakyReLU-9            [-1, 32, 32, 3]               0
Total params: 516,768
Trainable params: 516,768
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.21
Params size (MB): 1.97
Estimated Total Size (MB): 2.19
-------------------------------------------

In [27]:
# discriminator summary
summary(model_D, input_size=(1,2,200))

/mnt/scratch/rraj/venvs/py311/lib64/python3.11/site-packages/torch/nn/modules/conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /pytorch/aten/src/ATen/native/Convolution.cpp:1036.)
  return F.conv2d(


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 2, 200]             512
       BatchNorm2d-2           [-1, 64, 2, 200]             128
         LeakyReLU-3           [-1, 64, 2, 200]               0
           Dropout-4           [-1, 64, 2, 200]               0
            Conv2d-5          [-1, 128, 2, 200]          65,536
       BatchNorm2d-6          [-1, 128, 2, 200]             256
         LeakyReLU-7          [-1, 128, 2, 200]               0
           Dropout-8          [-1, 128, 2, 200]               0
            Conv2d-9          [-1, 256, 2, 200]         262,144
      BatchNorm2d-10          [-1, 256, 2, 200]             512
        LeakyReLU-11          [-1, 256, 2, 200]               0
          Dropout-12          [-1, 256, 2, 200]               0
           Conv2d-13          [-1, 512, 2, 200]       1,048,576
      BatchNorm2d-14          [-1, 512,